In [ ]:
!pip install -r requirements.txt --quiet

In [2]:
from utils import find_repo_root, load_dict, calculate_vectorized_correlation, get_fmri, get_pca, correlation_metric, download_fmri
from evaluation_utils import run_evaluation_pipeline

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
from sklearn.preprocessing import StandardScaler
import pickle
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from keras.regularizers import l2
from sklearn.model_selection import ParameterGrid
from tensorflow.keras.optimizers import Nadam

### Function Definitions

In [ ]:
def train_model(layer, sub, ROI, X_train, X_val, y_train, y_val, hp_combinations,  visualize_results = True):
    """
    conducts the training for a particular input layer of the CNN, subject & ROI. Saves model parameters & history
    :param X_train: training data (feature map PCs from first 800 videos from a particular layer)
    :param X_val: validation data (feature map PCs from videos 801-900 from a particular layer)
    :param y_train: training labels (scans for first 800 videos for the particular subject & ROI)
    :param y_val: training labels (scans for videos 801-900 for the particular subject & ROI)
    :param hp_combinations: dict with all parameter values in HPO. If hpo=False in run_training_pipeline, this only contains one single set of HP values
    :param visualize_results: whether training will be plotted
    :return: model parameters
    """
    print("y_train shape: ", y_train.shape)
    print("Y_val shape: ", y_val.shape)
    
    # apply training to every HP combination. If hpo == False, only one HP combination will be checked
    for hp_set in hp_combinations:
        
        # save all HP values of current iteration as variable
        num_hidden_layers = hp_set["num_hidden_layers"]
        l2_reg = hp_set["l2_reg"]
        learning_rate = hp_set["learning_rate"]
        dropout = hp_set["dropout"]
        num_epochs = hp_set["num_epochs"]
        
        # specify number of neurons per layer, depending on the number of inputs from a particular layer and number of output voxels
        # leads to symmetric funnel-like shape of the network, with increasing layer sizes for WB and decreasing size for all ROIs
        if num_hidden_layers == 1:
            input_neurons = X_train.shape[1]
            hidden1_neurons = y_train.shape[1] + (X_train.shape[1] - y_train.shape[1])*(1/2)
            output_neurons = y_train.shape[1]
        elif num_hidden_layers == 2:
            input_neurons = X_train.shape[1]
            hidden1_neurons = y_train.shape[1] + (X_train.shape[1] - y_train.shape[1])*(2/3)
            hidden2_neurons = y_train.shape[1] + (X_train.shape[1] - y_train.shape[1])*(1/3)
            output_neurons = y_train.shape[1]
        
        # model construction: 1 or 2 hidden layers. L2 Reg & Dropout.
        if num_hidden_layers == 1:
            model = Sequential([
                Dense(hidden1_neurons, input_shape=(input_neurons,),
                      activation='relu', kernel_regularizer=l2(l2_reg)),
                Dropout(dropout),
                Dense(output_neurons, activation='linear',
                      kernel_regularizer=l2(l2_reg)),
                Dropout(dropout)
            ])
        elif num_hidden_layers == 2:
            model = Sequential([
                Dense(hidden1_neurons, input_shape=(input_neurons,),
                      activation='relu', kernel_regularizer=l2(l2_reg)),
                Dropout(dropout),
                Dense(hidden2_neurons, activation='relu',
                      kernel_regularizer=l2(l2_reg)),
                Dropout(dropout),
                Dense(output_neurons, activation='linear',
                      kernel_regularizer=l2(l2_reg)),
                Dropout(dropout)
            ])
            
        # early stopping
        early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1)

        # Compiling the model
        nadam = Nadam(learning_rate=learning_rate)
        model.compile(optimizer=nadam, loss='mean_squared_error')
        
        # Training the model
        history = model.fit(X_train, y_train,
                            epochs=num_epochs,
                            batch_size=32,
                            validation_data=(X_val, y_val),
                            callbacks=[early_stopping],
                            verbose=1)
        
        if visualize_results:
            # print layer overview
            print(model.summary())
            
            # Plot training & validation loss values
            plt.plot(history.history['loss'])
            plt.plot(history.history['val_loss'])
            plt.title('Model loss')
            plt.ylabel('Loss')
            plt.xlabel('Epoch')
            plt.legend(['Train', 'Validation'], loc='upper left')
            plt.show()
        
        # save the model in models folder structure
        models_dir = os.path.join(os.getcwd(), "models", layer, ROI, sub)
        if not os.path.exists(models_dir):
            os.makedirs(models_dir)
        model_name = f"model_hidden_{num_hidden_layers}_lr_{learning_rate}_dropout_{dropout}_l2_{l2_reg}" + ".keras"
        model_path = os.path.join(models_dir, model_name)
        print("Saving model...")
        print(model_path)
        model.save(model_path)
        print("Model saved.")
        
        # save the model history
        history_dir = os.path.join(os.getcwd(), "model_histories", layer, ROI, sub)
        if not os.path.exists(history_dir):
            os.makedirs(history_dir)
        history_name = model_name.replace('.keras', '.history')
        history_path = os.path.join(history_dir, history_name)
        print("Saving training history...")
        with open(history_path, 'wb') as f:
            pickle.dump(history.history, f)
        print("Training history saved.")

In [6]:
def run_training_pipeline(hpo=False, import_type="direct"):
    """
    conducts training. Models will be saved in .keras format for evaluation.
    Parameters
    ----------
    hpo: if set to true, hyperparameter optimization will be conducted. Otherwise, a single set of hyperparameters will be applied. Adjust HP for that manually here.
    import_type: set to "direct" if the full PCA output from the main Resnet is directly loaded into the CWD.
                 set to "indirect" if a folder structure with one file per video is loaded.
    """
    # load one only one main PCA file into the Ucloud session. This will determine the layer
    layer_list = ["stage_1", "stage_2", "stage_3", "stage_4", "stage_5"]
    for i in layer_list:
        if os.path.exists(f"{i}_pca.pkl") or os.path.exists(f"{i}_pca.npy"):
            layer = i
            break

    # all subjects and ROIs to loop through. "WB" is "whole brain"
    subs = ["sub01","sub02","sub03","sub04","sub05","sub06","sub07","sub08","sub09","sub10"]
    ROIs = ["WB", "V1", "V2","V3", "V4", "LOC", "EBA", "FFA","STS", "PPA"]
    
    # unless already downloaded, download the fmri data
    download_fmri()
    
    # Load activations (PCA outputs)
    train_pca,val_pca = get_pca(layer, mode="val", import_type=import_type)
    
    # specify hyperparameter grid
    param_grid ={"learning_rate": [0.0001],
                "num_hidden_layers": [1,2],
                "dropout": [0.2, 0.4],
                "l2_reg":[0.001, 0.0001, 0.00001],
                "num_epochs": [100]}
    hp_combinations = ParameterGrid(param_grid)
    if hpo:
        # choose hp settings to test in current session
        hp_combinations = list(hp_combinations)[0:4]
    else:
        # if no HPO, specify hyperparameter settings manually here - one value per HP
        param_grid = {"learning_rate": [0.0001],
                     "num_hidden_layers": [2],
                     "dropout": [0.2],
                     "l2_reg":[0.0001],
                     "num_epochs": [100]}
        hp_combinations = ParameterGrid(param_grid)
        
    for sub in subs:
      for ROI in ROIs:
        print ("Starting ROI: ", ROI, "sub: ",sub)
        if ROI == "WB":
          track="full_track"
        else:
          track="mini_track"
        # Load fMRI data
        fmri_train, fmri_val = get_fmri(ROI, track, sub, mode="val")
    
        # model training
        train_model(layer=layer,
                    sub=sub,
                    ROI=ROI,
                    X_train=train_pca,
                    X_val=val_pca,
                    y_train=fmri_train,
                    y_val=fmri_val,
                    hp_combinations=hp_combinations,
                    visualize_results=True)
        
        print ("Completed ROI: ", ROI, "sub: ",sub)
        print("----------------------------------------------------------------------------")

### Run Training & Evaluation

In [ ]:
# conduct training & evaluation with validation set
if __name__ == "__main__":
    run_training_pipeline(hpo=True)
    run_evaluation_pipeline(data_mode="val")